## Load & Transform generic


In [122]:
# Taxi trips transformation codes 
# - Company master update 
# - Payment type master update 
# - Create a generic master update 
# - Update taxi trips with the most recent company master 
#   and payment type master codes 
# Weather transformation codes 

In [123]:
# Import data manipulation and visualization modules
from datetime import datetime
from dateutil.relativedelta import relativedelta
import pandas as pd
import requests
import os

import json

pd.set_option("display.max_columns", 30)


In [124]:
"""
AWS equivalents of the following operations in the current module:

1. Get the data from S3
2. taxi_trips transformation
3. update payment_type_master
4. update payment_type_master
5. update taxi_trips with payment_type and pament_type
   ids (replace string values ith ids)
6. weather data transformation
7. upload weather data to s3
8. upload taxi data to s3
9. upload the newest payment_type_master and company_master
"""


'\nAWS equivalents of the following operations in the current module:\n\n1. Get the data from S3\n2. taxi_trips transformation\n3. update payment_type_master\n4. update payment_type_master\n5. update taxi_trips with payment_type and pament_type\n   ids (replace string values ith ids)\n6. weather data transformation\n7. upload weather data to s3\n8. upload taxi data to s3\n9. upload the newest payment_type_master and company_master\n'

### Taxi trips transformation codes

In [125]:
# from datetime import datetime
# from dateutil.relativedelta import relativedelta

# Setting the date for 2 months ago
current_datetime = datetime.now() - relativedelta(months=2)
# Format the date
formatted_datetime = current_datetime.strftime("%Y-%m-%d")
# Construct the API URL
url = (
    f"https://data.cityofchicago.org/resource/ajtu-isnz.json?"
    f"$where=trip_start_timestamp >= '{formatted_datetime}T00:00:00' "
    f"AND trip_start_timestamp <= '{formatted_datetime}T23:59:59'&$limit=30000"
)
# Make the API request
response = requests.get(url)
# Process the response
data = response.json()


In [126]:
# Set up taxi_trips data
taxi_trips = pd.DataFrame(data)
taxi_trips.head()


,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,dropoff_community_area,fare,tips,tolls,extras,trip_total,payment_type,company,dropoff_centroid_latitude,dropoff_centroid_longitude,dropoff_centroid_location,pickup_community_area,pickup_centroid_latitude,pickup_centroid_longitude,pickup_centroid_location,pickup_census_tract,dropoff_census_tract
0,4e39f4210373d96df0c2792554f2f4b483a46f39,477e2191cd2213e2db9413d810936b2f24a73c8e527059...,2025-02-23T23:45:00.000,2025-02-24T00:00:00.000,830,0,33,13.25,0,0,0,15.75,Cash,Tac - Yellow Cab Association,41.857183858,-87.620334624,"{'type': 'Point', 'coordinates': [-87.62033462...",NaN,NaN,NaN,NaN,NaN,NaN
1,51d44b4e2e3cf13652298f7266c8420ac753e0c0,d4e172502a18df80190c19176a06c0d9344f71abdbe521...,2025-02-23T23:45:00.000,2025-02-24T00:00:00.000,682,4.61,6,17.17,0,0,0,17.67,Mobile,Globe Taxi,41.944226601,-87.655998182,"{'type': 'Point', 'coordinates': [-87.65599818...",8,41.899602111,-87.633308037,"{'type': 'Point', 'coordinates': [-87.63330803...",NaN,NaN
2,53cc0af124ad042946755076c289a0bb0b416e0f,0d3fc37b4e76ff16045a1e82cd817625cebff2683927eb...,2025-02-23T23:45:00.000,2025-02-24T00:00:00.000,1285,13.42,39,34,0,0,4,38,Cash,Taxicab Insurance Agency Llc,41.808916283,-87.596183344,"{'type': 'Point', 'coordinates': [-87.59618334...",56,41.79259236,-87.769615453,"{'type': 'Point', 'coordinates': [-87.76961545...",NaN,NaN
3,54c132ce98fd0dbdd522b518ab1022f59d0d2036,61d83eb1d34ccd8dac5bdd962bc13e2ac077b0c7ecbf1d...,2025-02-23T23:45:00.000,2025-02-24T00:00:00.000,1080,14.5,24,52.75,8,0,4,65.25,Credit Card,Globe Taxi,41.901206994,-87.676355989,"{'type': 'Point', 'coordinates': [-87.67635598...",76,41.980264315,-87.913624596,"{'type': 'Point', 'coordinates': [-87.91362459...",NaN,NaN
4,54c8e813447ccac84f1b1c526d782f3f045ae216,455b6b5cae6ca5a17cddd251485f2266d13d6a2c92f07c...,2025-02-23T23:45:00.000,2025-02-24T00:00:00.000,1052,15.53,7,38.75,8.85,0,5,53.1,Credit Card,Sun Taxi,41.922686284,-87.649488729,"{'type': 'Point', 'coordinates': [-87.64948872...",76,41.980264315,-87.913624596,"{'type': 'Point', 'coordinates': [-87.91362459...",NaN,NaN


In [127]:
# Drop columns
taxi_trips.drop(["pickup_census_tract", "dropoff_census_tract"], axis=1, inplace=True)
# Drop columns
taxi_trips.drop(
    ["pickup_centroid_location", "dropoff_centroid_location"], axis=1, inplace=True
)
# Drop rows with missing values
taxi_trips.dropna(inplace=True)
# Rename columns
taxi_trips.rename(
    columns={
        "pickup_community_area": "pickup_community_area_id",
        "dropoff_community_area": "dropoff_community_area_id",
    },
    inplace=True,
)
# Create a new column (datetime_for_weather), rounds down the datetime
# values to the nearest hour (removes minutes and seconds)
taxi_trips["datetime_for_weather"] = pd.to_datetime(
    taxi_trips["trip_start_timestamp"]
).dt.floor("h")


In [128]:
# Display taxi_trips
taxi_trips.head()


,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,dropoff_community_area_id,fare,tips,tolls,extras,trip_total,payment_type,company,dropoff_centroid_latitude,dropoff_centroid_longitude,pickup_community_area_id,pickup_centroid_latitude,pickup_centroid_longitude,datetime_for_weather
1,51d44b4e2e3cf13652298f7266c8420ac753e0c0,d4e172502a18df80190c19176a06c0d9344f71abdbe521...,2025-02-23T23:45:00.000,2025-02-24T00:00:00.000,682,4.61,6,17.17,0,0,0,17.67,Mobile,Globe Taxi,41.944226601,-87.655998182,8,41.899602111,-87.633308037,2025-02-23 23:00:00
2,53cc0af124ad042946755076c289a0bb0b416e0f,0d3fc37b4e76ff16045a1e82cd817625cebff2683927eb...,2025-02-23T23:45:00.000,2025-02-24T00:00:00.000,1285,13.42,39,34,0,0,4,38,Cash,Taxicab Insurance Agency Llc,41.808916283,-87.596183344,56,41.79259236,-87.769615453,2025-02-23 23:00:00
3,54c132ce98fd0dbdd522b518ab1022f59d0d2036,61d83eb1d34ccd8dac5bdd962bc13e2ac077b0c7ecbf1d...,2025-02-23T23:45:00.000,2025-02-24T00:00:00.000,1080,14.5,24,52.75,8,0,4,65.25,Credit Card,Globe Taxi,41.901206994,-87.676355989,76,41.980264315,-87.913624596,2025-02-23 23:00:00
4,54c8e813447ccac84f1b1c526d782f3f045ae216,455b6b5cae6ca5a17cddd251485f2266d13d6a2c92f07c...,2025-02-23T23:45:00.000,2025-02-24T00:00:00.000,1052,15.53,7,38.75,8.85,0,5,53.1,Credit Card,Sun Taxi,41.922686284,-87.649488729,76,41.980264315,-87.913624596,2025-02-23 23:00:00
5,560d7c2324cd34298f7f329101fdb79431d1d9cb,bedb270bce40664555c0e645a3f6c242d8f64ed541f6f5...,2025-02-23T23:45:00.000,2025-02-24T00:15:00.000,1153,12.47,5,32.25,0,0,6,38.25,Cash,Blue Ribbon Taxi Association,41.947791586,-87.683834942,76,41.980264315,-87.913624596,2025-02-23 23:00:00


In [129]:
# The same operations summarized in a separate function


def taxi_trips_transformations(taxi_trips: pd.DataFrame) -> pd.DataFrame:
    """Perform transformations with the taxi data
    Takes in a raw taxi trip DataFrame, cleans it, renames some columns,
    and creates a new timestamp column rounded to the hour (useful for
    weather joins and time-based analysis).

    Args:
    taxi_trips (pd.DataFrame): The dataframe holding the
        daily taxi trips

    Returns:
    pd.DataFrame
        The clened, transformed dataframe holding the
        daily taxi trips
    """

    if not isinstance(taxi_trips, pd.DataFrame):
        raise TypeError("taxi_trips is not a valid Pandas dataframe")

    taxi_trips.drop(
        [
            "pickup_census_tract",
            "dropoff_census_tract",
            "pickup_centroid_location",
            "dropoff_centroid_location",
        ],
        axis=1,
        inplace=True,
    )
    taxi_trips.dropna(inplace=True)
    taxi_trips.rename(
        columns={
            "pickup_community_area": "pickup_community_area_id",
            "dropoff_community_area": "dropoff_community_area_id",
        },
        inplace=True,
    )
    taxi_trips["datetime_for_weather"] = pd.to_datetime(
        taxi_trips["trip_start_timestamp"]
    ).dt.floor("h")

    return taxi_trips


### Company master update

In [130]:
# Retrieve company_master, set ID range and diplay the last 5 rows
company_master = taxi_trips["company"].drop_duplicates().reset_index(drop=True)
company_master = pd.DataFrame(
    {"company_id": range(1, len(company_master) + 1), "company": company_master}
)
company_master.tail()


,company_id,company
27,28,6574 - Babylon Express Inc.
28,29,U Taxicab
29,30,Tac - Blue Diamond Dispatch
30,31,2733 - 74600 Benny Jona
31,32,Metro Jet Taxi A.


In [131]:
# Create a new company data
new_company_data = [{"company": "Petani Cab Corp"}, {"company": "X"}, {"company": "Y"}]
new_company_mapping = pd.DataFrame(new_company_data)
new_company_mapping


,company
0,Petani Cab Corp
1,X
2,Y


In [132]:
# Get maximal ID for company_master
company_max_id = company_master["company_id"].max()
company_max_id


np.int64(32)

In [133]:
# Filter out companies that are in new_company_mapping but not in company_master
new_company_list = []
for company in new_company_mapping["company"].values:
    if company not in company_master["company"].values:
        # print(company)
        new_company_list.append(company)


# The previous filtering operation in list comprehended version (or in one line) 
new_companies_list = [
    company
    for company in new_company_mapping["company"].values
    if company not in company_master["company"].values
]

new_companies_list


['Petani Cab Corp', 'X', 'Y']

In [134]:
# New DataFrame for the new companies with unique company_ids starting right 
# after the current max master ID 
new_companies_df = pd.DataFrame(
    {
        "company_id": range(
            company_max_id + 1, company_max_id + 1 + len(new_companies_list)
        ),
        "company": new_companies_list,
    }
)
new_companies_df


,company_id,company
0,33,Petani Cab Corp
1,34,X
2,35,Y


In [135]:
# Add the new companies to the company_master
updated_company_master = pd.concat(
    [company_master, new_companies_df], ignore_index=True
)

updated_company_master.tail()


,company_id,company
30,31,2733 - 74600 Benny Jona
31,32,Metro Jet Taxi A.
32,33,Petani Cab Corp
33,34,X
34,35,Y


In [136]:
# The previous operations in standalone function version
def update_company_master(
    taxi_trips: pd.DataFrame, company_master: pd.DataFrame
) -> pd.DataFrame:
    """Extend the company master with new companies if thera are new
    companies.

    Parameters:
    taxi_trips: pd.DataFrame,
        Dataframe holding the daily taxi trips.
    company_master: pd.DataFrame
        Dataframe holding the company master data.

    Returns:
        _type_: _description_
        The updted company master data.
    """

    company_max_id = company_master["company_id"].max()
    new_companies_list = [
        company
        for company in taxi_trips["company"].values
        if company not in company_master["company"].values
    ]

    new_companies_df = pd.DataFrame(
        {
            "company_id": range(
                company_max_id + 1, company_max_id + 1 + len(new_companies_list)
            ),
            "company": new_companies_list,
        }
    )

    updated_company_master = pd.concat(
        [company_master, new_companies_df], ignore_index=True
    )

    return updated_company_master


In [137]:
# Create a test company dataset and verify that the update_company_master 
# function works correctly
taxi_trips_company_only = pd.DataFrame(
    {
        "company_id": [1, 2, 3],
        "company": ["Petani Cab Corp", "X", "Y"],
    }
)
updated_company_master = update_company_master(
    taxi_trips_company_only,
    company_master
)
updated_company_master.tail()


,company_id,company
30,31,2733 - 74600 Benny Jona
31,32,Metro Jet Taxi A.
32,33,Petani Cab Corp
33,34,X
34,35,Y


### Payment type master update

In [138]:
# Retrieve the unique payment types from the taxi_trips
payment_type_master = (
    taxi_trips["payment_type"].drop_duplicates().reset_index(drop=True)
)

# Transform unique payment types into a proper master table
payment_type_master = pd.DataFrame(
    {
        "payment_type_id": range(1, len(payment_type_master) + 1),
        "payment_type": payment_type_master,
    }
)

# Create payment types test dataset and display it
taxi_trips_payment_type_only = pd.DataFrame(
    {
        "payment_type_id": [1, 2, 3],
        "payment_type": ["Credit Card", "X", "Y"],
    }
)
taxi_trips_payment_type_only


,payment_type_id,payment_type
0,1,Credit Card
1,2,X
2,3,Y


In [139]:
# Create a payment type update standalone function
def update_payment_type_master(
    taxi_trips: pd.DataFrame, payment_type_master: pd.DataFrame
) -> pd.DataFrame:
    """Extend the payment_type master with new companies if thera are new
    companies.

    Parameters:
    taxi_trips: pd.DataFrame,
        Dataframe holding the daily taxi trips.
    payment_type_master: pd.DataFrame
        Dataframe holding the payment_type master data.

    Returns:
        updated_payment_type_master: 
        Dataframe of the updated payment_type master data.
    """

    payment_type_max_id = payment_type_master["payment_type_id"].max()
    new_payment_types_list = [
        payment_type
        for payment_type in taxi_trips["payment_type"].values
        if payment_type not in payment_type_master["payment_type"].values
    ]

    new_payment_type_df = pd.DataFrame(
        {
            "payment_type_id": range(
                payment_type_max_id + 1,
                payment_type_max_id + 1 + len(new_payment_types_list),
            ),
            "payment_type": new_payment_types_list,
        }
    )

    updated_payment_type_master = pd.concat(
        [payment_type_master, new_payment_type_df], ignore_index=True
    )

    # updated_payment_type_master = 1
    return updated_payment_type_master


In [140]:
# Display payment type test dataset
taxi_trips_payment_type_only


,payment_type_id,payment_type
0,1,Credit Card
1,2,X
2,3,Y


In [141]:
# Verify that the update_payment_type_master works correctly
updated_payment_type_master = update_payment_type_master(
    taxi_trips_payment_type_only,
    payment_type_master
)
updated_payment_type_master.tail()

,payment_type_id,payment_type
4,5,Unknown
5,6,No Charge
6,7,Dispute
7,8,X
8,9,Y


### Create a generic master update

In [142]:
# Create a generic master update function
def update_master(
    taxi_trips: pd.DataFrame, master: pd.DataFrame, id_column: str, value_column: str
) -> pd.DataFrame:
    """Extend the master with new values if there are any.

    Parameters:
    taxi_trips: pd.DataFrame,
        Dataframe holding the daily taxi trips.
    payment_type_master: pd.DataFrame
        Dataframe holding the master master data.
    id_column: str
        The id column of the master database
    value_column: str
        name of the column in master_df containing the values


    Returns:
        _type_: _description_
        The updated master data.
    """

    max_id = master[id_column].max()
    new_values_list = list(
        set(taxi_trips[value_column].values) - set(master[value_column].values)
    )

    # print(new_values_list)
    new_values_df = pd.DataFrame(
        {
            id_column: range(max_id + 1, max_id + 1 + len(new_values_list)),
            value_column: new_values_list,
        }
    )

    updated_master = pd.concat([master, new_values_df], ignore_index=True)

    # updated_payment_type_master = 1
    return updated_master


In [143]:
# Verify that the update_master works correctly on payment types
test_payment_type_master = update_master(
    taxi_trips=taxi_trips_payment_type_only,
    master=payment_type_master,
    id_column="payment_type_id",
    value_column="payment_type",
)


In [144]:
# Display payment_type_master
payment_type_master

,payment_type_id,payment_type
0,1,Mobile
1,2,Cash
2,3,Credit Card
3,4,Prcard
4,5,Unknown
5,6,No Charge
6,7,Dispute


In [145]:
# Display taxi_trips_payment_type_only
taxi_trips_payment_type_only


,payment_type_id,payment_type
0,1,Credit Card
1,2,X
2,3,Y


In [146]:
# Display resulted payment_type_master
test_payment_type_master


,payment_type_id,payment_type
0,1,Mobile
1,2,Cash
2,3,Credit Card
3,4,Prcard
4,5,Unknown
5,6,No Charge
6,7,Dispute
7,8,Y
8,9,X


In [147]:
# Verify that the update_master works correctly on companies
test_company_master = update_master(
    taxi_trips=taxi_trips_company_only,
    master=company_master,
    id_column="company_id",
    value_column="company",
)
test_company_master

,company_id,company
0,1,Globe Taxi
1,2,Taxicab Insurance Agency Llc
2,3,Sun Taxi
3,4,Blue Ribbon Taxi Association
4,5,Taxi Affiliation Services
5,6,Chicago Independents
6,7,Flash Cab
7,8,City Service
8,9,5 Star Taxi
9,10,Medallion Leasin


### Update taxi_trips with the most recent company_master <br> and payment_type master codes 

In [148]:
# Display taxi_trips to be updated
taxi_trips.head()


,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,dropoff_community_area_id,fare,tips,tolls,extras,trip_total,payment_type,company,dropoff_centroid_latitude,dropoff_centroid_longitude,pickup_community_area_id,pickup_centroid_latitude,pickup_centroid_longitude,datetime_for_weather
1,51d44b4e2e3cf13652298f7266c8420ac753e0c0,d4e172502a18df80190c19176a06c0d9344f71abdbe521...,2025-02-23T23:45:00.000,2025-02-24T00:00:00.000,682,4.61,6,17.17,0,0,0,17.67,Mobile,Globe Taxi,41.944226601,-87.655998182,8,41.899602111,-87.633308037,2025-02-23 23:00:00
2,53cc0af124ad042946755076c289a0bb0b416e0f,0d3fc37b4e76ff16045a1e82cd817625cebff2683927eb...,2025-02-23T23:45:00.000,2025-02-24T00:00:00.000,1285,13.42,39,34,0,0,4,38,Cash,Taxicab Insurance Agency Llc,41.808916283,-87.596183344,56,41.79259236,-87.769615453,2025-02-23 23:00:00
3,54c132ce98fd0dbdd522b518ab1022f59d0d2036,61d83eb1d34ccd8dac5bdd962bc13e2ac077b0c7ecbf1d...,2025-02-23T23:45:00.000,2025-02-24T00:00:00.000,1080,14.5,24,52.75,8,0,4,65.25,Credit Card,Globe Taxi,41.901206994,-87.676355989,76,41.980264315,-87.913624596,2025-02-23 23:00:00
4,54c8e813447ccac84f1b1c526d782f3f045ae216,455b6b5cae6ca5a17cddd251485f2266d13d6a2c92f07c...,2025-02-23T23:45:00.000,2025-02-24T00:00:00.000,1052,15.53,7,38.75,8.85,0,5,53.1,Credit Card,Sun Taxi,41.922686284,-87.649488729,76,41.980264315,-87.913624596,2025-02-23 23:00:00
5,560d7c2324cd34298f7f329101fdb79431d1d9cb,bedb270bce40664555c0e645a3f6c242d8f64ed541f6f5...,2025-02-23T23:45:00.000,2025-02-24T00:15:00.000,1153,12.47,5,32.25,0,0,6,38.25,Cash,Blue Ribbon Taxi Association,41.947791586,-87.683834942,76,41.980264315,-87.913624596,2025-02-23 23:00:00


In [149]:
# Define this update as a standalone function
def update_taxi_trips_with_master_data(
    taxi_trips: pd.DataFrame,
    payment_type_master: pd.DataFrame,
    company_master: pd.DataFrame,
) -> pd.DataFrame:
    """
    Update the taxi_trips DataFrame by replacing the string columns
    for payment_type and company with their respective IDs from the
    master dataframes. The function drops the original string columns
    for payment_type and company.

    Args:
        taxi_trips (pd.DataFrame):
        A DataFrame containing taxi trip records with `payment_type`
        and `company` columns.
        payment_type_master (pd.DataFrame):
        A DataFrame that maps payment type descriptions to payment type IDs.
        company_master (pd.DataFrame):
        A DataFrame that maps company names to company IDs.

    Returns:
        pd.DataFrame:
        The updated taxi_trips DataFrame with `payment_type_id` and
        `company_id` columns, and the original string columns removed.
    """

    taxi_trips_id = taxi_trips.merge(payment_type_master, on="payment_type")

    taxi_trips_id = taxi_trips_id.merge(company_master, on="company")

    taxi_trips_id.drop(["payment_type", "company"], axis=1, inplace=True)

    return taxi_trips_id


In [150]:
# Display taxi_trips:
# taxi_trips_id.sample(5)


In [151]:
# Update payment_type and company string columns in the taxi_trips
# DataFrame with their corresponding ID values from the payment_type_master
# and company_master DataFrames, respectively.
taxi_trips_id = update_taxi_trips_with_master_data(
    taxi_trips=taxi_trips,
    payment_type_master=payment_type_master,
    company_master=company_master,
)


### Weather transformation codes

In [152]:
# Standalone function to complete weather transformation
def transform_weather_data(weather_data: json) -> pd.DataFrame:
    """
    Make transformations on the daily weather api response.
    It transform raw weather data from an API (in JSON format) into a clean,
    structured pandas DataFrame, suitable for analysis or storage.

    Args:
        weather_data (json):
        The daily weather data from the open Meteo API.

    Returns:
        pd.DataFrame:
        A Dataframe representetion of the weather data.
    """

    weather_data_filtered = {
        "datetime": weather_data["hourly"]["time"],
        "tempretaure": weather_data["hourly"]["temperature_2m"],
        "wind_speed": weather_data["hourly"]["wind_speed_10m"],
        "rain": weather_data["hourly"]["rain"],
        "precipitation": weather_data["hourly"]["precipitation"],
    }

    weather_df = pd.DataFrame(weather_data_filtered)

    weather_df["datetime"] = pd.to_datetime(weather_df["datetime"])

    weather_df.to_csv("weather_data_date.csv", index=False)

    return weather_df


In [153]:
# from datetime import datetime
# from dateutil.relativedelta import relativedelta
#
# An API request to the Open-Meteo weather service, querying historical
# weather data for a specific location (latitude: 41.85, longitude: -87.65),
# which corresponds to Chicago, IL. It retrieves hourly data for temperature,
# wind speed, rain, and precipitation for a specific date that is two months
# prior to the current date.
current_datetime = datetime.now() - relativedelta(months=2)
formatted_datetime = current_datetime.strftime("%Y-%m-%d")
url = "https://archive-api.open-meteo.com/v1/era5"
params = {
    "latitude": 41.85,
    "longitude": -87.65,
    "start_date": formatted_datetime,
    "end_date": formatted_datetime,
    "hourly": "temperature_2m,wind_speed_10m,rain,precipitation",
}
response = requests.get(url, params=params)

# Parse the JSON response from the API, which contains the weather data for
# the specified date.
weather_data = response.json()

# Processes the raw JSON data (weather_data) and transforms it into a
# pandas DataFrame.
weather_data_df = transform_weather_data(weather_data)
weather_data_df


,datetime,tempretaure,wind_speed,rain,precipitation
0,2025-02-23 00:00:00,-2.3,12.0,0.0,0.0
1,2025-02-23 01:00:00,-2.0,10.1,0.0,0.0
2,2025-02-23 02:00:00,-2.7,9.9,0.0,0.0
3,2025-02-23 03:00:00,-3.2,10.7,0.0,0.0
4,2025-02-23 04:00:00,-3.7,10.6,0.0,0.0
5,2025-02-23 05:00:00,-4.1,10.4,0.0,0.0
6,2025-02-23 06:00:00,-4.4,9.6,0.0,0.0
7,2025-02-23 07:00:00,-4.6,9.4,0.0,0.0
8,2025-02-23 08:00:00,-4.8,10.0,0.0,0.0
9,2025-02-23 09:00:00,-5.1,10.2,0.0,0.0
